Using textgenrnn on tensorflow backend to generate some poetry. This model can be used letter- or word-based. I used it word-based.
I found this model on a youtube video by Max Woolf
https://www.youtube.com/watch?v=RW7mP6BfZuY&list=PLEVLQU6Zt5WCI19mIj3Cmg0sWmZ48Gr5b&index=4

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Setting the model according to recomendations of author

In [ ]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 8,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 30,   # set higher to train the model for longer
    'gen_epochs': 50,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

Importing the data

In [ ]:
file_name = "Goethe_all.txt"
model_name = 'Goethe_all'   # change to set file name of resulting trained models/texts

Training the model

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell Bidirectional LSTMs
Training on 469,618 word sequences.

Epoch 1/30
458/458 [==============================] - 86s 188ms/step - loss: 6.0609
Epoch 2/30
458/458 [==============================] - 78s 171ms/step - loss: 5.0841
Epoch 3/30
458/458 [==============================] - 78s 170ms/step - loss: 4.5510
Epoch 4/30
458/458 [==============================] - 78s 171ms/step - loss: 4.0522
Epoch 5/30
458/458 [==============================] - 78s 171ms/step - loss: 3.5744
Epoch 6/30
458/458 [==============================] - 79s 173ms/step - loss: 3.1233
Epoch 7/30
458/458 [==============================] - 78s 171ms/step - loss: 2.7110
Epoch 8/30
458/458 [==============================] - 79s 172ms/step - loss: 2.3503
Epoch 9/30
458/458 [==============================] - 78s 171ms/step - loss: 2.0356
Epoch 10/30
458/458 [==============================] - 79s 173ms/step - loss: 1.7645
Epoch 11/30
458/458 [==============================] - 78s 171

Saving the model

In [ ]:
textgen.save("Goethe.model2")

Generating the text and downloading it with adjustabel Temperature/Creativity and legth

[Linktext](https://)You can download a large amount of generated text from your model with the cell below! Rerun the cell as many times as you want for even more text!

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = 0.5  
prefix = None  # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

A interactive mode is possible in which the model shows you the n most likely following words and you can pick from them

In [ ]:
textgen.generate(interactive=True, top_n=5)

Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: 

	2: ;
	3: ,
	4: wie
	5: .

Progress: 

Your choice?
> 4
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: .
	2: 

	3: sei
	4: ,
	5: ?

Progress: e

Your choice?
> 5
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: 

	2: ,
	3: «
	4: »ein
	5: male

Progress: e ?

Your choice?
> 4
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: der
	2: ;
	3: .
	4: 

	5: ?

Progress: e ? »ein

Your choice?
> 1
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: ,
	2: schnee
	3: 

	4: menschen
	5: freunde

Progress: e ? »ein der

Your choice?


KeyboardInterrupt: ignored